In [ ]:
# %matplotlib inline #notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것 

import numpy
import pandas
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

#setting the seed for reproducibility
#seed = 10110
seed = 12061204
numpy.random.seed(seed)

#폴더의 파일목록 불러오기
import os
path_dir = './(input)Deepchem toxcast_data all/first_input'
file_list = os.listdir(path_dir)
input_files = []
i = 0

for item in file_list :
        input_files.append(item)
        #print(item)
        
        
def create_deep_learning_model():
    model = Sequential()
    #첫번째 인자 : 출력 뉴런의 수 , input_dim : 입력 뉴런의 수, activation : 활성화 함수.
    model.add(Dense(2048, input_dim=2048, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model    

def main(input_name):
    dataframe = pandas.read_csv('./(input)Deepchem toxcast_data all/first_input/'+input_name)
    dataframe2 = pandas.read_csv('note.csv')
#     print(dataframe.head)
    mols = []
    fps = []
    i = 0 #전체 물질 개수
    j = 0 #type 이 1인 물질 개수 즉 ligand개수
    
    for index, row in dataframe.iterrows():
        mol = Chem.MolFromSmiles(row['SMILES'])
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
        mols.append(mol)
        fps.append(fp)
        i += 1
        if(row["Type"]==1) : 
            j += 1
    
    np_fps = []
    for fp in fps:
        arr = numpy.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        np_fps.append(arr)
        
    np_fps_array = numpy.array(np_fps)
    
#     my_string = input_name
#     Type = my_string.split('(')[0]
    enc_y = dataframe['Type']
    
    sm = SMOTE(random_state=12, ratio = 'minority')
    x_train_res, y_train_res = sm.fit_sample(np_fps_array, enc_y) #결과데이터

    # evaluate model with standardized dataset
    #estimator = KerasClassifier(build_fn=create_deep_learning_model, nb_epoch=100, batch_size=5, verbose=0)
    estimator = KerasClassifier(build_fn=create_deep_learning_model, nb_epoch=100, batch_size=5)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    #results = cross_val_score(estimator, np_fps_array, enc_y, cv=kfold)
    results = cross_val_score(estimator, x_train_res, y_train_res, cv=kfold)
    #https://datascienceschool.net/view-notebook/266d699d748847b3a3aa7b9805b846ae/
    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

    y_pred = cross_val_predict(estimator, x_train_res, y_train_res, cv=kfold)
    conf_mat = confusion_matrix(y_train_res, y_pred)
    conf_mat


    model = create_deep_learning_model()
    history = model.fit(x_train_res, y_train_res, epochs=5, batch_size=5) #histroy 객체를 만환한다 이것을 이용해서  각 단계에서의 acc와 loss값을 알 수 있다
#     plt.plot(history.history['acc'])
#     print(history.history['acc'])
#     plt.show()


    
    Acc3 = repr(results.mean())
    Acc4 = Acc3[2:4] + "." + Acc3[4:6]
    
    Acc = repr(history.history['acc'][-1])
    if(Acc == '1.0') :
        Acc2 = "100"
        model.save("./trained_models/First_27_models/"+input_name+'_model' + '(' + Acc2 + ')'+' (' + Acc4 + ')'+'.h5')
    else :
        Acc2 = Acc[2:4] + "." + Acc[4:6]
        
        print(Acc2)
        model.save("./trained_models/First_27_models/"+input_name+'_model' + '(' + Acc2 + ')' +' (' + Acc4 + ')'+'.h5')

    
    #모델정보 정리를 위한 코드.
    Total_Chemicals = i
    Ligand = j
    Non_ligand = i - j
    Model_accuacy = Acc2
    info_list = [Total_Chemicals, Ligand, Non_ligand, Model_accuacy ]
    dataframe2[input_name.replace("(input).csv","")] = info_list
    dataframe2.to_csv("./trained_models/info_input_data,model_acc_first.csv", mode='w')
    

if __name__ == "__main__":
    for input_name in input_files :
        main(input_name)    #d우선 모델 빨리 만들어보려고 이걸로.
    